# Get SOTA Representation For Tiles
In this notebook, we'll embed the training tiles and testing tiles in 5 different SOTA algorithms, evaluate their performance on 3 classifiers and then apply some visualisation to the results

**This notebook is intended to be run on Google Colab**

In [ ]:
# For Step 1. Dataloader
import numpy as np
import os
import torch
from time import time
from torch.autograd import Variable
import sys

# For Step 2. Embedding
from img2vec_pytorch import Img2Vec
import numpy as np
import os
from PIL import Image
from matplotlib import cm
from time import time
import torch

# For Step 3. Classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# For Step 4. Visualisation
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DIR = '/content/drive/MyDrive/tile2vec/'

In [ ]:
tile2vec_dir = '/tile2vec'
sys.path.append('../')
sys.path.append(DIR)
from src.tilenet import make_tilenet
from src.resnet import ResNet18

Unzip TOA Train Tiles

In [ ]:
!unzip /content/drive/MyDrive/toa_train_tiles.zip > /dev/null

Unzip TOA Test Tiles

In [ ]:
!unzip /content/drive/MyDrive/toa_test_tiles.zip > /dev/null

Unzip Vis Tiles

In [ ]:
!unzip /content/drive/MyDrive/cross_section_tiles.zip > /dev/null

## Step 1. Loading trained/pre-trained Tile2Vec model
In this step, we will initialize a new TileNet model and then load the pre-trained weights.

In [ ]:
# Setting up model
in_channels = 3
z_dim = 512
cuda = torch.cuda.is_available()
tilenet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
# Use old model for now
tilenet = ResNet18() # Comment out this line if using trained Tile2Vec
if cuda: tilenet.cuda()
print (cuda)

In [ ]:
# Uncomment below line if using trained Tile2Vec model
#model_fn = '../models/TileNet_epoch99_toa_data.ckpt'

# Uncomment below line if using pre-trained Tile2Vec model
model_fn = '../models/naip_trained.ckpt'
checkpoint = torch.load(model_fn, map_location=torch.device('cpu'))
tilenet.load_state_dict(checkpoint)
tilenet.eval()

## Step 2. Embed tiles
In this step, we'll use TileNet to embed the NAIP tiles provided in `tile2vec/data/tiles`. There are 1000 tiles in total, named `1tile.npy` through `1000tile.npy`.

In [ ]:
# Point to directory holding the tiles
#tile_dir = '../data/toa_train_tiles'
tile_dir = '../data/toa_test_tiles/'

# The number of tiles in the directory
# Train = 58498
# Test = 19981
n_tiles = 19981

# Load the label vector
y_test = np.load(os.path.join(tile_dir, 'y.npy'))
print(y_test.shape)

## Tile2Vec

In [ ]:
# Embed tiles
t0 = time()
X_t2v = np.zeros((n_tiles, z_dim))
for idx in range(n_tiles):
    tile = np.load(os.path.join(tile_dir, '{}tile.npy'.format(idx))) #used to bbe idx+1 so undo if error
    # Get first 4 ≠ 3 NAIP channels (5th is CDL mask)
    tile = tile[:,:,:4]
    # Rearrange to PyTorch order
    tile = np.moveaxis(tile, -1, 0)
    tile = np.expand_dims(tile, axis=0)
    # Scale to [0, 1]
    #tile = tile / 255
    # Embed tile
    
    # Add 4th feature to each pixel for ResNet structure
    new_col = np.full((1,1,51, 51), 0)
    tile = np.append(tile, new_col, axis=1)
    
    tile = torch.from_numpy(tile).float()
    tile = Variable(tile)
    
    if cuda: tile = tile.cuda()
        
    z = tilenet.encode(tile)
    
    if cuda: z = z.cpu()
        
    z = z.data.numpy()
    X_t2v[idx,:] = z
    
t1 = time()
print('Embedded {} time: {:0.3f}s'.format(n_tiles, t1-t0))

## ResNet 18

In [ ]:
!pip install img2vec_pytorch

In [ ]:
# Initialize Img2Vec with GPU
img2vec = Img2Vec(cuda=True)
UPSCALE = False
t0 = time()
X_res18 = np.zeros((n_tiles, z_dim))
for idx in range(n_tiles):

    # Read in an image
    tile = np.load(os.path.join(tile_dir, '{}tile.npy'.format(idx)))

    # Upscale image
    if (UPSCALE):
        x, y, z = tile.shape

        tile = resize(tile, (224, 224,z))

    im = Image.fromarray((tile * 255).astype(np.uint8))   
    
    vec = img2vec.get_vec(im, tensor=True)
    
    new_vec = [x for x in vec[0]]

    X_res18[idx, :] = new_vec
    
    if (idx % 1000 == 0):
        print ("Progress: {:.2f}%".format(idx/n_tiles*100))
        
t1 = time()
print('Embedded {} tiles: {:0.3f}s'.format(n_tiles, t1-t0))

## SwAV

In [ ]:
model = torch.hub.load('facebookresearch/swav', 'resnet50')
model.eval()
model.cuda()

In [ ]:
# Embed tiles
t0 = time()
z_dim = 1000
UPSCALE = True
X_SWAV = np.zeros((n_tiles, z_dim))
for idx in range(n_tiles):
    tile = np.load(os.path.join(tile_dir, '{}tile.npy'.format(idx))) #used to bbe idx+1 so undo if error
    # Get first 4 ≠ 3 NAIP channels (5th is CDL mask)
    tile = tile[:,:,:4]
    # Rearrange to PyTorch order
    tile = np.moveaxis(tile, -1, 0)
    tile = np.expand_dims(tile, axis=0)
    # Scale to [0, 1]
    #tile = tile / 255
    # Embed tile

    # Upscale image
    if (UPSCALE):

        x, y, z, w = tile.shape

        tile = resize(tile, (x,y,224,224))

    tile = torch.from_numpy(tile).float()
    tile = Variable(tile)
    
    if cuda: tile = tile.cuda()

    z = model(tile)
    
    if cuda: z = z.cpu()
        
    z = z.data.numpy()
    X_SWAV[idx,:] = z
    
    if idx%100 == 0:
        print ("Progress: {:.2f}".format(idx/n_tiles * 100))
        
t1 = time()
print('Embedded {} time: {:0.3f}s'.format(n_tiles, t1-t0))

## AlexNet

In [ ]:
# Initialize Img2Vec with GPU
img2vec = Img2Vec(model = 'alexnet', cuda=True)


t0 = time()
X_alex = np.zeros((n_tiles, 4096))

for idx in range(n_tiles):

    # Read in an image
    tile = np.load(os.path.join(tile_dir, '{}tile.npy'.format(idx)))
    
    im = Image.fromarray((tile * 255).astype(np.uint8))   
    
    vec = img2vec.get_vec(im, tensor=True)
    
    new_vec = [x for x in vec[0]]

    X_alex[idx, :] = new_vec
    
    if (idx % 1000 == 0):
        print ("Progress: {:.2f}%".format(idx/n_tiles*100))
        
t1 = time()
print('Embedded {} tiles: {:0.3f}s'.format(n_tiles, t1-t0))

## Step 3. Evaluate Performance On Classifiers

Load the embeddings that will be evaluated

In [ ]:
X_train = np.load(os.path.join('../embeddings/', 'X_SwAV_train.npy'))
X_test = np.load(os.path.join('../embeddings/', 'X_SwAV_test.npy'))

y_train = np.load(os.path.join('../embeddings/', 'y_train.npy'))
y_test = np.load(os.path.join('../embeddings/', 'y_test.npy'))

Split test set into a test and validation set

In [ ]:
X_tst, X_val, y_tst, y_val = train_test_split(X_test, y_test, test_size=0.2)

## Decision Tree

Determine best depth for decision tree

In [ ]:
depths = list(range(1, 41))

X_tst, X_val, y_tst, y_val = train_test_split(X_test, y_test, test_size=0.2)
accs = []
for depth in range(1,41):
    print (depth)
    decisionTree = DecisionTreeClassifier(random_state=0, max_depth = depth)

    decisionTree.fit(X_train, y_train)

    predictions = decisionTree.predict(X_val)
    
    correct_preds = 0
    for i in range(0,len(predictions)):
        if predictions[i] == y_val[i]:
            correct_preds += 1

    accs.append(correct_preds/len(predictions))

print (np.where(accs == np.amax(accs)))
print (np.amax(accs))

plt.plot(depths, accs) #adds the line
plt.ylabel('Accuracy') #xlabel
plt.xlabel('Depth') #ylabel
plt.title('Accuracy of Decision Tree as Depth Increases')
plt.savefig('depthvsAccuracyDT') # Best = 7
plt.show()


In [ ]:
np.save(os.path.join('../embeddings/', 'DT_accs_swav.npy'), accs)

Best depths

resnet18 = 7

tile2vec (trained) = 8

tile2vec (pre-trained) = 4

AlexNet = 6

SwAV = 7

Print decision tree accuracy maps:

In [ ]:
accs_alexnet = np.load(os.path.join('../embeddings/', 'DT_accs_alexnet.npy'))
accs_resnet18 = np.load(os.path.join('../embeddings/', 'DT_accs_resnet18.npy'))
accs_tile2vec = np.load(os.path.join('../embeddings/', 'DT_accs_tile2vec.npy'))
accs_PTtile2vec = np.load(os.path.join('../embeddings/', 'DT_accs_pre_tile2vec.npy'))
accs_swav = np.load(os.path.join('../embeddings/', 'DT_accs_swav.npy'))
depths = list(range(1, 41))

In [ ]:
plt.style.use('ggplot')

plt.plot(depths, accs_alexnet, '-gD', color="blue",  mfc='yellow', markevery=[5], label="AlexNet") #adds the line
plt.plot(depths, accs_resnet18,'-gD', color="red", mfc='yellow', markevery=[6], label="ResNet18") #adds the line
plt.plot(depths, accs_tile2vec,'-gD', color="green", mfc='yellow',  markevery=[7], label="Tile2Vec") #adds the line
plt.plot(depths, accs_PTtile2vec,'-gD', color="pink", mfc='yellow',  markevery=[3], label="Tile2Vec (pre-trained)") #adds the line
plt.plot(depths, accs_swav,'-gD', color="orange", mfc='yellow',  markevery=[6], label="SwAV") #adds the line

plt.plot([0], [0.75], color="yellow") #adds the line


plt.ylabel('Accuracy') #xlabel
plt.xlabel('Depth') #ylabel
plt.savefig('depthvsAccuracyDT') # Best = 7
plt.legend()
plt.savefig('../figures/modelDTAccuracy.png')
plt.show()



Get performance of decision tree with best hyper-parameters 

In [ ]:
n_trials = 1
accs = np.zeros((n_trials,))
precisions = np.zeros((n_trials,))
recalls = np.zeros((n_trials,))
fscores = np.zeros((n_trials,))
for i in range(n_trials):
    dt = DecisionTreeClassifier(random_state=0, max_depth =7)
    dt.fit(X_train, y_train)
    accs[i] = dt.score(X_tst, y_tst)
    y_pred = dt.predict(X_test)
    y_pred_prob = dtrf.predict_proba(X_test)
    precisions[i], recalls[i], fscores[i], _ = precision_recall_fscore_support(y_test, y_pred, average='macro')
    print ("TRIAL: {}".format(i))
    
print ("____Accuracy____")
print('Mean accuracy: {:0.4f}'.format(accs.mean()))
print('Standard deviation: {:0.4f}'.format(accs.std()))

print ("____Macro Precision____")
print('Mean macro precision: {:0.4f}'.format(precisions.mean()))
print('Standard deviation: {:0.4f}'.format(precisions.std()))

print ("____Macro Recall____")
print('Mean macro recall: {:0.4f}'.format(recalls.mean()))
print('Standard deviation: {:0.4f}'.format(recalls.std()))

print ("____Macro F1-Score____")
print('Mean macro F1-Score: {:0.4f}'.format(fscores.mean()))
print('Standard deviation: {:0.4f}'.format(fscores.std()))




## Multinomial Logistic Regression


In [ ]:
n_trials = 1
accs = np.zeros((n_trials,))
precisions = np.zeros((n_trials,))
recalls = np.zeros((n_trials,))
fscores = np.zeros((n_trials,))

for i in range(n_trials):
    
    # Splitting data and training RF classifer
    # Define the multinomial logistic regression model    
    lr = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, solver='saga', multi_class='multinomial'))
    
    # Fit the model on the whole dataset
    lr.fit(X_train, y_train)
    
    accs[i] = lr.score(X_tst, y_tst)
    
    y_pred = lr.predict(X_tst)
        
    precisions[i], recalls[i], fscores[i], _ = precision_recall_fscore_support(y_tst, y_pred, average='macro')
    
    print ("TRIAL: {}".format(i))
    
print ("____Accuracy____")
print('Mean accuracy: {:0.4f}'.format(accs.mean()))
print('Standard deviation: {:0.4f}'.format(accs.std()))

print ("____Macro Precision____")
print('Mean macro precision: {:0.4f}'.format(precisions.mean()))
print('Standard deviation: {:0.4f}'.format(precisions.std()))

print ("____Macro Recall____")
print('Mean macro recall: {:0.4f}'.format(recalls.mean()))
print('Standard deviation: {:0.4f}'.format(recalls.std()))

print ("____Macro F1-Score____")
print('Mean macro F1-Score: {:0.4f}'.format(fscores.mean()))
print('Standard deviation: {:0.4f}'.format(fscores.std()))

## Random Forest Classifier

**Get best number of estimators**

In [ ]:

accs = []
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]

train_results = []
test_results = []

for estimator in n_estimators:
    print (estimator)
    rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)

    rf.fit(X_train, y_train)

    predictions = rf.predict(X_val)

    correct_preds = 0
    for i in range(0,len(predictions)):
        if predictions[i] == y_val[i]:
            correct_preds += 1

    accs.append(correct_preds/len(predictions))

print (np.where(accs == np.amax(accs)))
print (np.amax(accs))

plt.plot(n_estimators, accs) #adds the line
plt.ylabel('Accuracy') #xlabel
plt.xlabel('Number of Estimators') #ylabel
plt.savefig('EstimatorsvsAccuracyRF') # Best = 7
plt.show()

In [ ]:
plt.style.use('ggplot')
plt.plot(n_estimators, accs) #adds the line
plt.ylabel('Accuracy') #xlabel
plt.xlabel('Number of Estimators') #ylabel
plt.savefig('EstimatorsvsAccuracyRF') # Best = 7
plt.show()

In [ ]:
np.save(os.path.join('../embeddings/', 'RF_embs_accs_t2v.npy'), accs)

In [ ]:
accs_alexnet = np.load(os.path.join('../embeddings/', 'RF_embs_accs_alex.npy'))
accs_resnet18 = np.load(os.path.join('../embeddings/', 'RF_embs_accs_res18.npy'))
accs_tile2vec = np.load(os.path.join('../embeddings/', 'RF_embs_accs_t2v.npy'))
accs_PTtile2vec = np.load(os.path.join('../embeddings/', 'RF_embs_accs_pre_t2v.npy'))
accs_swav = np.load(os.path.join('../embeddings/', 'RF_embs_accs_swav.npy'))
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]


In [ ]:
plt.style.use('ggplot')

plt.plot(n_estimators, accs_alexnet, '-gD', color="blue",  mfc='yellow', markevery=[7], label="AlexNet") #adds the line
plt.plot(n_estimators, accs_resnet18,'-gD', color="red", mfc='yellow', markevery=[5], label="ResNet18") #adds the line
plt.plot(n_estimators, accs_tile2vec,'-gD', color="green", mfc='yellow',  markevery=[8], label="Tile2Vec") #adds the line
plt.plot(n_estimators, accs_PTtile2vec,'-gD', color="pink", mfc='yellow',  markevery=[8], label="Tile2Vec (pre-trained)") #adds the line
plt.plot(n_estimators, accs_swav,'-gD', color="orange", mfc='yellow',  markevery=[6], label="SwAV") #adds the line

plt.plot([0], [0.75], color="yellow") #adds the line


plt.ylabel('Accuracy') #xlabel
plt.xlabel('Depth') #ylabel
plt.savefig('depthvsAccuracyDT') # Best = 7
plt.legend()
plt.savefig('../figures/modelRFEstsAccuracy.png')
plt.show()

**Get optimal max depth**

In [ ]:
max_depths = np.linspace(1, 41, 41, endpoint=True)

accs = []

train_results = []
test_results = []

for depth in max_depths:
    
    print (depth)
    
    rf = RandomForestClassifier(max_depth=depth, n_jobs=-1)

    rf.fit(X_train, y_train)

    predictions = rf.predict(X_val)

    correct_preds = 0
    for i in range(0,len(predictions)):
        if predictions[i] == y_val[i]:
            correct_preds += 1

    accs.append(correct_preds/len(predictions))

print (np.where(accs == np.amax(accs)))
print (np.amax(accs))

plt.plot(max_depths, accs) #adds the line
plt.ylabel('Accuracy') #xlabel
plt.xlabel('Depth') #ylabel
plt.savefig('DepthvsAccuracyRF') # Best = 7
plt.show()

In [ ]:
np.save(os.path.join('../embeddings/', 'RF_depths_accs_alex.npy'), accs)

In [ ]:
accs_alexnet = np.load(os.path.join('../embeddings/', 'RF_depths_accs_alex.npy'))
accs_resnet18 = np.load(os.path.join('../embeddings/', 'RF_depths_accs_res18.npy'))
accs_tile2vec = np.load(os.path.join('../embeddings/', 'RF_depths_accs_tile2vec.npy'))
accs_PTtile2vec = np.load(os.path.join('../embeddings/', 'RF_depths_accs_pre_t2v.npy'))
accs_swav = np.load(os.path.join('../embeddings/', 'RF_depths_accs_swav.npy'))
depths = list(range(1, 42))

In [ ]:
plt.style.use('ggplot')

plt.plot(depths, accs_alexnet, '-gD', color="blue",  mfc='yellow', markevery=[27], label="AlexNet") #adds the line
plt.plot(depths, accs_resnet18,'-gD', color="red", mfc='yellow', markevery=[19], label="ResNet18") #adds the line
plt.plot(depths, accs_tile2vec,'-gD', color="green", mfc='yellow',  markevery=[22], label="Tile2Vec") #adds the line
plt.plot(depths, accs_PTtile2vec,'-gD', color="pink", mfc='yellow',  markevery=[16], label="Tile2Vec (pre-trained)") #adds the line
plt.plot(depths, accs_swav,'-gD', color="orange", mfc='yellow',  markevery=[28], label="SwAV") #adds the line

plt.plot([0], [0.75], color="yellow") #adds the line


plt.ylabel('Accuracy') #xlabel
plt.xlabel('Depth') #ylabel
plt.savefig('depthvsAccuracyDT') # Best = 7
plt.legend()
plt.savefig('../figures/modelRFDepthAccuracy.png')
plt.show()

                Depth     Estimators
    Tile2Vec     23          200
    Tile2Vec(PT) 17          200
    ResNet18     20          32
    AlexNet      28          100
    SWAV         29          64

Get average performance of Random Forest using optimal hyper-parameters over 10 trials

In [ ]:
n_trials = 10
accs = np.zeros((n_trials,))
precisions = np.zeros((n_trials,))
recalls = np.zeros((n_trials,))
fscores = np.zeros((n_trials,))
for i in range(n_trials):
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    accs[i] = rf.score(X_tst, y_tst)
    y_pred = rf.predict(X_tst)
    y_pred_prob = rf.predict_proba(X_tst)
    precisions[i], recalls[i], fscores[i], _ = precision_recall_fscore_support(y_tst, y_pred, average='macro')
    print ("TRIAL: {}".format(i))
    
print ("____Accuracy____")
print('Mean accuracy: {:0.4f}'.format(accs.mean()))
print('Standard deviation: {:0.4f}'.format(accs.std()))

print ("____Macro Precision____")
print('Mean macro precision: {:0.4f}'.format(precisions.mean()))
print('Standard deviation: {:0.4f}'.format(precisions.std()))

print ("____Macro Recall____")
print('Mean macro recall: {:0.4f}'.format(recalls.mean()))
print('Standard deviation: {:0.4f}'.format(recalls.std()))

print ("____Macro F1-Score____")
print('Mean macro F1-Score: {:0.4f}'.format(fscores.mean()))
print('Standard deviation: {:0.4f}'.format(fscores.std()))



Save the best model for future classifications

In [ ]:
pickle.dump(full_rf, open('../models/full_rf.sav', 'wb'))

## Step 4. Visualise The Embeddings

**Plot t-SNE**

For an NDVI t-SNE plot the file ```ViewTSNE.py``` must be used to get the NDVI score of each embedding which will then be used as the ```y``` term with a green to white colour palette

In [ ]:
X_full = np.load(os.path.join('../embeddings/', 'X_SwAV_train.npy'))
y_full = np.load(os.path.join('../embeddings/', 'X_SwAV_test.npy'))

X_subset = X_full[::3]
y_subset = y_full[::3]

In [ ]:
time_start = time.time()

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)

tsne_results = tsne.fit_transform(X_subset)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start)) # 4618 seconds

In [ ]:
df_subset = pd.DataFrame()
df_subset['y'] = y_subset
df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 3),
    data=df_subset,
    legend="full",
    alpha=0.3
)

plt.savefig('/content/drive/MyDrive/tile2vec/figures/tsne_toa_alexnet_full')